In [1]:
import spacy

In [2]:
!python -m spacy download en_core_web_lg

2023-07-13 18:03:39.277453: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [3]:
!python -m spacy download en

2023-07-13 18:04:21.355150: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 42.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
nlp = spacy.load("en_core_web_lg")
nlp

In [5]:
doc = nlp("Donad Trump was President of USA")

In [6]:
doc

Donad Trump was President of USA

In [7]:
type(doc)

spacy.tokens.doc.Doc

In [8]:
doc.ents

(Donad Trump, USA)

In [9]:
doc.ents[0],type(doc.ents[0])

(Donad Trump, spacy.tokens.span.Span)

In [10]:
from spacy import displacy
displacy.render(doc,style="ent",jupyter=True  )

In [11]:
fact='When Facebook (United States) announced its plans to acquire WhatsApp in February 2014, WhatsApps founders attached a purchase price of $16 billion: $4 billion in cash and $12 billion remaining in Facebook shares. This price tag is dwarfed by the actual price Facebook paid: $21.8 billion, or $55 per user'

In [12]:
doc1=nlp(fact)

In [13]:
doc1

When Facebook (United States) announced its plans to acquire WhatsApp in February 2014, WhatsApps founders attached a purchase price of $16 billion: $4 billion in cash and $12 billion remaining in Facebook shares. This price tag is dwarfed by the actual price Facebook paid: $21.8 billion, or $55 per user

In [14]:
doc1.ents
# ents-entities in documnet

(Facebook,
 United States,
 WhatsApp,
 February 2014,
 WhatsApps,
 $16 billion,
 $4 billion,
 $12 billion,
 Facebook,
 Facebook,
 $21.8 billion,
 55)

In [15]:
for ent in doc1.ents:
  print(ent.text,ent.start_char,ent.end_char,ent.label_)

# label_ --means classified class or entity

Facebook 5 13 ORG
United States 15 28 GPE
WhatsApp 61 69 ORG
February 2014 73 86 DATE
WhatsApps 88 97 ORG
$16 billion 136 147 MONEY
$4 billion 149 159 MONEY
$12 billion 172 183 MONEY
Facebook 197 205 ORG
Facebook 260 268 ORG
$21.8 billion 275 288 MONEY
55 294 296 MONEY


In [16]:

# ORG-Comapnies,Agencies,institutions
# GPE-Geopolitical entity i.e. country,state,chain_no_types
# MONEY
# DATE

In [17]:
import json
with open('payment_terms_.json','rb') as f:
  data=json.load(f)

In [18]:
data['examples'][0]

{'id': 783,
 'text': "(c)mrbwill pay undisputed amounts within 30 days after receiving Supplier's invoice.",
 'meta': {},
 'annotation_approver': None,
 'labels': [[41, 48, 'Payment Term']]}

In [19]:
data['examples'][0].keys()

dict_keys(['id', 'text', 'meta', 'annotation_approver', 'labels'])

In [20]:
data['examples'][0]['text']

"(c)mrbwill pay undisputed amounts within 30 days after receiving Supplier's invoice."

In [21]:
data['examples'][9]['labels']

[[115, 122, 'Payment Term']]

In [22]:
data['examples'][10]['meta']

{}

In [23]:
training_data=[]
for example in data['examples']:
  temp_dict = {}
  temp_dict['text']=example['text']
  temp_dict['entities']=[]
  temp_dict['entities']=example['labels']
  training_data.append(temp_dict)

In [24]:
len(training_data)

89

In [25]:
training_data[0]['text']

"(c)mrbwill pay undisputed amounts within 30 days after receiving Supplier's invoice."

In [26]:
training_data[67]['entities']

[[72, 92, 'Payment Term'], [188, 203, 'Payment Term']]

In [27]:
training_data[0]['text'][41:48]

'30 days'

In [28]:
training_data[10]['entities']

[]

In [29]:
# there are some missing entities are there so we need to validate this data

In [30]:
def validate(l):
  if len(l)==0:
    return 0
  else: return 1


In [31]:
train_valid_data=[]
for i in range(len(training_data)):
  if validate(training_data[i]['entities']):
    train_valid_data.append(training_data[i])

In [32]:
len(train_valid_data)

29

In [33]:
from spacy.tokens import DocBin  #DocBin serialize the data (convert data obj to format that allow us to store or transmit data)
from tqdm import tqdm            #provides a fast and extensible progress bar for loops and iterables
nlp=spacy.blank('en')            #create blank spacy model for English
doc_bin=DocBin()                 #create new docbin obj to hold serialized annotations

In [34]:
from spacy.util import filter_spans   #function takes a sequence of Span objects and removes duplicates or overlaps.
for training_ex in tqdm(train_valid_data):
  text=training_ex['text']
  labels=training_ex['entities']
  doc=nlp.make_doc(text)               # creates a Doc object from the text using the blank model. This will only segment the text into tokens and assign them hashes. No other linguistic annotations will be added. explain in simple words
  ents=[]
  for start,end,label in labels:
    span=doc.char_span(start,end,label=label,alignment_mode='contract')
    if span is None:

      print('Skipping entity')
    else:
      ents.append(span)
      # print(span)
  filtered_ents=filter_spans(ents)  #filters the list of spans and removes any duplicates or overlaps.
  doc.ents=filtered_ents           # assigns the filtered spans to the ents attribute of the doc.
  # print(doc)
  doc_bin.add(doc)               #adds the doc’s annotations to the DocBin for serialization.
doc_bin.to_disk('train.spacy')   #Saving the DocBin to disk:

100%|██████████| 29/29 [00:00<00:00, 586.10it/s]


In [35]:
# docs = list(doc_bin.get_docs(nlp.vocab))
# docs

# this is for unserialize obj

In [36]:
!python -m spacy init fill-config base_config.cfg config.cfg

# This is a command to initialize a config file for Spacy training.
# It takes a base config file and fills in the missing values with defaults
#  or suggestions based on the data and pipeline. It outputs a new config file
#  that can be used for training

2023-07-13 18:04:37.401918: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [42]:
!python -m spacy train config1.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

# This is a command to train a Spacy pipeline using a config file.
# It takes the config file and specifies the output directory and the
#  paths to the training and development data. It will train the pipeline
#   according to the settings in the config file and save the best model to
#   the output directory.

2023-07-13 18:14:43.888726: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-07-13 18:14:47,054] [INFO] Set up nlp object from config
[2023-07-13 18:14:47,069] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-07-13 18:14:47,073] [INFO] Created vocabulary
[2023-07-13 18:14:49,816] [INFO] Added vectors: en_core_web_lg
[2023-07-13 18:14:49,818] [INFO] Finished initializing nlp object
[2023-07-13 18:14:50,621] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     31.50    0.00    0.00    0.00    0.

In [38]:
ner=spacy.load('model-best')

In [39]:
doc_n=ner('I will deposit payment within 10 days')
color={'PAYMENT TERM':'#F78DE3'}
opt={'colors':color}
spacy.displacy.render(doc_n,style='ent',options=opt,jupyter=True)

/usr/local/lib/python3.10/dist-packages/spacy/displacy/__init__.py:215: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [40]:
doc = ner("i have given you 30 thousand rupees, i am expecting it back with 10 weeks ")

colors = {"PAYMENT TERM": "#F67DE3"}
options = {'ents':['PAYMENT TERM'],"colors": colors}

# spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

displacy.render(doc,style='ent',jupyter=True)